In [ ]:
import numpy as np
import requests
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Embedding
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

# Download dataset from GitHub
url = 'https://raw.githubusercontent.com/dwyl/english-words/master/words.txt'
response = requests.get(url)
text = response.text.lower()

# Prepare the dataset
seq_length = 100
sequences = []

for i in range(seq_length, len(text)):
    seq = text[i-seq_length:i]
    sequences.append(seq)

# Map characters to integers
chars = sorted(list(set(text)))
char_to_int = {c: i for i, c in enumerate(chars)}
int_to_char = {i: c for i, c in enumerate(chars)}

# Create training data
X = np.zeros((len(sequences), seq_length, len(chars)))
y = np.zeros((len(sequences), len(chars)))

for i, seq in enumerate(sequences):
    for t, char in enumerate(seq):
        X[i, t, char_to_int[char]] = 1
    y[i, char_to_int[text[i]]] = 1

# Build LSTM model
model = Sequential()
model.add(LSTM(128, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Dense(len(chars), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.01))

# Train the model
model.fit(X, y, epochs=20, batch_size=64)

# Function to generate text
def generate_lstm_text(seed_text, next_words=100):
    result = seed_text
    for _ in range(next_words):
        # Encode the text to integers
        encoded = np.zeros((1, seq_length, len(chars)))
        for t, char in enumerate(seed_text[-seq_length:]):
            encoded[0, t, char_to_int[char]] = 1
        # Predict the next character
        prediction = model.predict(encoded, verbose=0)
        predicted_char_index = np.argmax(prediction)
        predicted_char = int_to_char[predicted_char_index]
        result += predicted_char
        seed_text += predicted_char
    return result

# Example generation
seed_text = "Artificial intelligence is"
generated_text = generate_lstm_text(seed_text)
print(generated_text)
